In [ ]:
%install '.package(path: "$cwd/YOLOv4")' YOLOv4
%install '.package(url: "https://github.com/tensorflow/swift-models", .branch("tensorflow-0.9"))' ModelSupport
import TensorFlow
import YOLOv4
import ModelSupport
import Foundation

In [2]:
let imageURL = URL(fileURLWithPath: "dog_512.jpg")
let img = Image(jpeg: imageURL)
var im = img.tensor.reshaped(to: [1,512,512,3]) / 255
var yolov4 = YOLOv4<Float>(cls:80).load(fileAtPath: "yolov4.weights")
let y = (im |> yolov4)//.transposed(permutation: [0,3,1,2])

In [8]:
var y: Tensor<Float> = im |> yolov4
var boxes: [Bbox] = load(from: y)
var x: [Bbox] = nms(bboxes: boxes, thresh: 0.25, thresh_iou: 0.25)

In [15]:
print(x)

[YOLOv4.Bbox(x1: 0.20858431, x2: 0.75025344, y1: 0.023745755, y2: 0.023772418, score: 0.9904691, cls: [1.1263836e-05, 1.2374674e-06, 0.14665926, 1.5147727e-05, 2.8237404e-05, 0.0016123652, 3.7147824e-05, 0.9511117, 2.8722563e-05, 6.149019e-06, 4.7358863e-06, 1.6540173e-06, 7.282404e-06, 0.00013044477, 3.0164824e-08, 1.748036e-07, 3.4246042e-07, 6.10557e-07, 1.8980951e-06, 7.164894e-06, 1.2373471e-06, 5.189633e-07, 4.296885e-06, 2.4346923e-07, 1.2127759e-06, 3.7241591e-06, 3.6874117e-06, 5.486054e-08, 1.5624842e-05, 2.3811113e-07, 2.2101915e-06, 3.9515865e-07, 6.1647637e-07, 2.0258217e-06, 2.1611843e-06, 7.4445046e-07, 1.1315058e-06, 3.9023845e-07, 6.1345077e-07, 3.192519e-06, 2.2118203e-07, 7.3223746e-05, 1.7924795e-06, 5.402734e-07, 1.3239295e-06, 4.0158234e-06, 8.174586e-08, 5.7768816e-08, 1.2499163e-06, 2.9577055e-07, 7.659379e-07, 5.871997e-08, 4.2087814e-07, 6.76109e-08, 2.6476087e-07, 5.5141845e-06, 3.472651e-05, 9.620725e-06, 1.2298759e-05, 2.182033e-06, 3.1168638e-06, 5.012771e